# Follower network
We start from the Twitter follower network constructed for the paper [Right and left, partisanship predicts (asymmetric) vulnerability to misinformation](http://doi.org/10.37016/mr-2020-55). The following data files are available at https://doi.org/10.7910/DVN/6CZHH5:
* `anonymized-friends.json` 
* `measures.tab`

Briefly, this network was constructed as follows:
* We collected all tweets containing links (URLs) from a 10% random sample of public posts between June 1 and June 30, 2017, through the Twitter Decahose. 
* We selected all accounts that shared at least ten links from a set of news sources with known political valence (Bakshy et al., 2015). 
* We further selected those who shared at least one link from a source labeled as low-quality (https://github.com/BigMcLargeHuge/opensources). 
* We excluded likely bot accounts according to the BotometerLite classifier (Yang et al., 2020).

We keep the nodes with both partisanship and misinformation attributes, then we take the core of the network with approximately 10k nodes, and finally remove a random sample of edges to preserve the original average in/out-degree (number of friends/followers).

In [39]:
import networkx as nx
import csv
import json
import random
import importlib
import bot_model

In [26]:
path = "../EmpiricalNet_followers/"

In [13]:
# File has 3 columns: ID \t partisanship \t misinformation \n
partisanship = {}
misinformation = {}
with open(path + "measures.tab") as fd:
    rd = csv.reader(fd, delimiter="\t")
    next(rd) # skip header row
    for row in rd:
        partisanship[int(row[0])] = row[1]
        misinformation[int(row[0])] = row[2]

In [14]:
with open(path + 'anonymized-friends.json') as fp:
    adjlist = json.load(fp)

In [15]:
G = nx.DiGraph() 

In [16]:
# Directed network follower -> friend
for s in adjlist:
    n = int(s)
    if n in partisanship and n in misinformation:
        G.add_node(n, party=partisanship[n], misinfo=misinformation[n]) 
        for f in adjlist[s]:
            G.add_edge(n,f)

In [17]:
average_friends = G.number_of_edges() / G.number_of_nodes()
print("{} nodes and {} edges initially, with average number of friends {}".format(G.number_of_nodes(), G.number_of_edges(), average_friends))
friends = nx.subgraph(G, partisanship.keys())
print("{} nodes and {} edges after filtering".format(friends.number_of_nodes(), friends.number_of_edges()))

58048 nodes and 10499218 edges initially, with average number of friends 180.87131339581038
15056 nodes and 4327448 edges after filtering


In [19]:
# k-core decomposition until ~ 10k nodes in core
core_number = nx.core_number(friends)
nodes = friends.number_of_nodes()
k = 0
while nodes > 10000:
    k_core = nx.k_core(friends, k, core_number)
    nodes = k_core.number_of_nodes()
    k += 10
while nodes < 10000:
    k_core = nx.k_core(friends, k, core_number)
    nodes = k_core.number_of_nodes()
    k -= 1
print("{}-core has {} nodes, {} edges".format(k, k_core.number_of_nodes(), k_core.number_of_edges()))

94-core has 10006 nodes, 4144687 edges


In [20]:
# the network is super dense, so let us delete a random sample of edges
# we can set the initial average in/out-degree (average_friends) as a target 
friends_core = k_core.copy()
edges_to_keep = int(friends_core.number_of_nodes() * average_friends)
edges_to_delete = friends_core.number_of_edges() - edges_to_keep
deleted_edges = random.sample(friends_core.edges(), edges_to_delete)
friends_core.remove_edges_from(deleted_edges)
print("{}-core after edge-sampling has {} nodes, {} edges, and average number of friends {}".format(k, friends_core.number_of_nodes(), friends_core.number_of_edges(), friends_core.number_of_edges() / friends_core.number_of_nodes()))

94-core after edge-sampling has 10006 nodes, 1809798 edges, and average number of friends 180.8712772336598


In [22]:
nx.write_gml(friends_core, path + 'follower_network.gml')

## Simulations

In [52]:
# test
importlib.reload(bot_model)
follower_net = bot_model.init_net(False, verbose=True, human_network = path + 'follower_network.gml', beta=0.01, gamma=0.001)
avg_quality = bot_model.simulation(False, network=follower_net, verbose=True, mu=0.5, phi=1, alpha=15)
print('average quality for follower network:', avg_quality)

Reading human network...
Generating bot network...
Merging human and bot networks...
Humans following bots...
time_steps =  0 , q =  200
time_steps =  1 , q =  0.4801671070272221
time_steps =  2 , q =  0.4931137924415558
average quality for follower network: 0.49038907607976123


In [29]:
# experiments 
# params:    alpha, beta, gamma, phi, flooding
# targeting: hubs, partisans, conservatives, misinfo spreaders, core